# GAN Implementation
Author: Taylor Qin<br>
Version: v1-14APR

In [3]:
import numpy as np
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Input
import matplotlib.pyplot as plt

# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model

if __name__ == '__main__':
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    model = define_discriminator()
    plot_model(model, to_file='discriminator_plot.png', show_shapes=True, show_layer_names=True)

    
#     # Prepocess
#     x_train = x_train.astype('float32') / 255.
#     x_test = x_test.astype('float32') / 255.
#     x_train = x_train.reshape((x_train.shape[0], -1))
#     x_test = x_test.reshape((x_test.shape[0], -1))

#     # Train
#     encoder, autoencoder = train(x_train=x_train)
#     encode_images = encoder.predict(x_test)
#     decode_images = autoencoder.predict(x_test)
#     plot_images(decode_images, x_test)